<a href="https://colab.research.google.com/github/chanderbijlani/Coursera_Capstone/blob/main/The_Battle_of_Neighbourhoods_WEEk_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Battle of Neighbourhoods (Week-2)

---
## Applied Data Science Capstone project (IBM/Coursera)
By Chander Bijlani


## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction/Business Problem <a name="introduction"></a>

## Problem Description 
### Delhi, the capital of india is known for its food culture and rich lifestyle.It is influenced by the food habits of its residents and is where Mughlai cuisine originated.It includes a variety of international cuisines which are popular among the residents along with Indian cuisine.
### So the aim of this project is to find a suitable location in delhi for a person who is looking to open his restaurant so that it gets people attention and can earn good profit to survive this competitive market and also for food delivering companies to look for popular food restaurants.
## Background Discussion
### Delhi ranks 62 in list of world’s best cities. The dearth of food habits among the Delhi's residents created a unique style of cooking which became popular throughout the world, with dishes such as Kebab, biryani, tandoori. The city's classic dishes include butter chicken, dal makhani, shahi paneer, aloo chaat, chaat, dahi bhalla, kachori, gol gappe, samosa, chole bhature, chole kulche, gulab jamun, jalebi and lassi.
## Stakeholders/Target Audience
### This detailed analysis can be used by a person to find the food hotspots in delhi for opening his/her restaurants or by food delivering companies to look for popular food restaurants.

## Data <a name="data"></a>

### The data that can be used to solve our problem or the factors that can influence to solve our problem are:


1.   Delhi Neighbourhoods dataset from kaggle : 
This [dataset](https://https://www.kaggle.com/shaswatd673/delhi-neighborhood-data?select=restaurant_dataSet.csv) contains information about neighbourhoods in the delhi such as their name, borough, location, etc.
2.   Creating a location dataset using foursqaure api which will give us the most common venues to establish/open our restaurant.
3. Using Folium library to create interactive visulaization maps of popular or most common venues.


Importing required libraries

In [ ]:
import pandas as pd 
import numpy as np 
import folium
import requests

In [ ]:
from google.colab import files
uploaded = files.upload()



Saving delhi_dataSet.csv to delhi_dataSet.csv


In [ ]:
# df = pd.read_csv('restaurant_dataSet.csv', error_bad_lines=False)
# df.head()

In [ ]:
df = pd.read_csv('delhi_dataSet.csv', error_bad_lines=False)
df.head()

,Unnamed: 0,Borough,Neighborhood,latitude,longitude
0,0,North West Delhi,Adarsh Nagar,28.614192,77.071541
1,1,North West Delhi,Ashok Vihar,28.699453,77.184826
2,2,North West Delhi,Azadpur,28.707657,77.175547
3,3,North West Delhi,Bawana,28.799660,77.032885
4,4,North West Delhi,Begum Pur,NaN,NaN


## Cleaning the data/ Data Preprocessing

In [ ]:
df =  df.drop(['Unnamed: 0'],axis=1)
df = df.dropna()
df.reset_index(inplace=True,drop=True)
df.rename(columns={'latitude': 'Latitude','longitude':'Longitude','Neighborhood':'Neighbourhood'}, inplace=True)
# df.rename(columns={'Neighborhood':'Neighbourhood'}, inplace=True)
df

,Borough,Neighbourhood,Latitude,Longitude
0,North West Delhi,Adarsh Nagar,28.614192,77.071541
1,North West Delhi,Ashok Vihar,28.699453,77.184826
2,North West Delhi,Azadpur,28.707657,77.175547
3,North West Delhi,Bawana,28.799660,77.032885
4,North West Delhi,Dhaka,39.031714,-90.261223
...,...,...,...,...
158,West Delhi,Rajouri Garden,28.642152,77.116060
159,West Delhi,Shivaji Place,28.651657,77.121703
160,West Delhi,Tilak Nagar,28.639650,77.094039
161,West Delhi,Vikas Nagar,28.644009,77.054470


### Printing the rows of dataframe

In [ ]:
df.shape

(163, 4)

So our dataframe consists of 163 rows and 4 columns

## Creating a map of Delhi using latitude and longitudes

In [ ]:
delhimap = folium.Map(location=[28.7041,77.1025],zoom_start=12)

for lat,lng,borough,neighbourhood in zip(df['Latitude'],df['Longitude'],df['Borough'],df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(delhimap)
delhimap

## Defining FourSquare Api Credentails

In [ ]:
Client_id = '# your client id'
Client_secret = '# your client secret'
version = '20180604'

### Now we are getting the top 100 venues in each neighbourhood in Delhi using foursqaure api

In [ ]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            Client_id, 
            Client_secret, 
            version, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
delhi_venues = getNearbyVenues(names=df['Neighbourhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )


Adarsh Nagar
Ashok Vihar
Azadpur
Bawana
Dhaka
Jahangirpuri
Karala
Keshav Puram
Kingsway Camp
Kohat Enclave
Model Town
Narela
Pitam Pura
Rani Bagh
Rithala
Shalimar Bagh
Shakti Nagar
Bara Hindu Rao
Chandni Chowk
Chawri Bazaar
Civil Lines
Dariba Kalan
Kamla Nagar
Kashmiri Gate
Kotwali
Lahori Gate
Nehru Vihar
Paharganj
Pratap Nagar
Pul Bangash
Roshanara Bagh
Sangam Vihar
Sant Nagar
Sarai Kale Khan
Sarai Rohilla
Shakti Nagar
Shastri Nagar
Timarpur
Wazirabad
Babarpur
Dilshad Garden
Kabir Nagar
Mandoli
Maujpur
Nand Nagri
Naveen Shahdara
Shahdara
Sonia Vihar
Shastri Park
Yamuna Vihar
Chandni Chowk
Daryaganj
Jhandewalan
Karol Bagh
Bara Hindu Rao
Paharganj
Barakhamba Road
Chanakyapuri
Connaught Place
Gautampuri
Gole Market
Golf Links
INA Colony
Khan Market
Pragati Maidan
Brij Puri
East Vinod Nagar
Gandhi Nagar
Geeta Colony
Jhilmil Colony
Krishna Nagar
Laxmi Nagar
Mayur Vihar
Pandav Nagar
Patparganj
Preet Vihar
Anand Vihar
Shakarpur
Shastri Nagar
Vasundhara Enclave
Vishwas Nagar
Vivek Vihar
Shahd

In [ ]:
print(delhi_venues.shape)
delhi_venues.head()

(1145, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Adarsh Nagar,28.614192,77.071541,Bikanerwala,28.613391,77.076084,Indian Restaurant
1,Ashok Vihar,28.699453,77.184826,malwa,28.701699,77.185399,Farm
2,Ashok Vihar,28.699453,77.184826,Grand Plaza,28.696577,77.185245,Hotel Bar
3,Ashok Vihar,28.699453,77.184826,ELIXIR FERTILITY CENTRE,28.700034,77.188093,Health & Beauty Service
4,Ashok Vihar,28.699453,77.184826,Bakers Stop,28.700495,77.188716,Bakery


### Getting the venues for each neighbourhood

In [ ]:
delhi_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Adarsh Nagar,1,1,1,1,1,1
Alaknanda,12,12,12,12,12,12
Anand Vihar,7,7,7,7,7,7
Ashok Vihar,7,7,7,7,7,7
Azadpur,3,3,3,3,3,3
...,...,...,...,...,...,...
Vasundhara Enclave,4,4,4,4,4,4
Vikaspuri,3,3,3,3,3,3
Vishwas Nagar,2,2,2,2,2,2


### Analyzing each neighbourhood in Delhi

In [ ]:
#Using One-hot encoding
delhi_onehot = pd.get_dummies(delhi_venues[["Venue Category"]],prefix="",prefix_sep="")
# add neighborhood column back to dataframe
delhi_onehot["Neighbourhood"] = delhi_venues["Neighbourhood"]
# move neighbourhood column to be the first column
fix_column = [delhi_onehot.columns[-1]] + list(delhi_onehot[:-1])
delhi_onehot = delhi_onehot[fix_column]
delhi_onehot.head()

,Neighbourhood,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Bike Shop,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Burger Joint,Burmese Restaurant,Bus Station,Cafeteria,Café,Camera Store,Campground,Castle,Chinese Restaurant,Clothing Store,Coffee Shop,Comedy Club,...,Public Art,Rental Car Location,Restaurant,Road,Sake Bar,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Sculpture Garden,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Stables,Stadium,Steakhouse,Supplement Shop,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Wine Bar,Women's Store,Yoga Studio,Neighbourhood
0,Adarsh Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Adarsh Nagar
1,Ashok Vihar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Ashok Vihar
2,Ashok Vihar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Ashok Vihar
3,Ashok Vihar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Ashok Vihar
4,Ashok Vihar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Ashok Vihar


In [ ]:
delhi_onehot = delhi_onehot.T.drop_duplicates().T
delhi_onehot


,Neighbourhood,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Bike Shop,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Burger Joint,Burmese Restaurant,Bus Station,Cafeteria,Café,Camera Store,Campground,Castle,Chinese Restaurant,Clothing Store,Coffee Shop,Comedy Club,...,Pub,Public Art,Rental Car Location,Restaurant,Road,Sake Bar,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Sculpture Garden,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Stables,Stadium,Steakhouse,Supplement Shop,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Wine Bar,Women's Store,Yoga Studio
0,Adarsh Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Ashok Vihar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Ashok Vihar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Ashok Vihar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Ashok Vihar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1140,Tilak Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1141,Tilak Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1142,Vikaspuri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1143,Vikaspuri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# delhi_onehot['Neighbourhood'] = pd.to_numeric(delhi_onehot['Neighbourhood'])
# delhi_venue_grouped = delhi_onehot.groupby('Neighbourhood').mean().reset_index()
# delhi_venue_grouped


## Create a function that returns the most common venues

In [ ]:
def most_common_venues(row,top_venues):
   row_categories = row.iloc[1:]
   row_categories_sorted =  row_categories.sort_values(ascending=False)
   return row_categories_sorted.index.values[0:top_venues]

In [ ]:
# Display the top 10 venues of the neighbourhood 
top_venues = 10
indicators = ['st','nd','rd']
# create columns according to number of top venues
columns = ['Neighbourhood']
for i in np.arange(top_venues):
  try:
    columns.append('{}{} Most Common Venue'.format(i+1,indicators[i]))
  except:
    columns.append('{}th Most Common Venue'.format(i+1))

neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = delhi_onehot['Neighbourhood']

for i in np.arange(delhi_onehot.shape[0]):
  neighbourhoods_venues_sorted.iloc[i, 1:] = most_common_venues(delhi_onehot.iloc[i, :],top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adarsh Nagar,Indian Restaurant,Yoga Studio,Flea Market,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop
1,Ashok Vihar,Farm,Yoga Studio,Flower Shop,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop
2,Ashok Vihar,Hotel Bar,Yoga Studio,Flea Market,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop
3,Ashok Vihar,Health & Beauty Service,Yoga Studio,Flower Shop,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop
4,Ashok Vihar,Bakery,Yoga Studio,Food,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court


## Kmeans Clustering for clustering neighbourhoods in Delhi

In [ ]:
from sklearn.cluster import KMeans
#Taking 5 Clusters
k_clusters = 5
# removing unnecessary feature
delhi_group_clustering = delhi_onehot.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=k_clusters, random_state=0).fit(delhi_group_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 0, 0, 0, 0, 0, 0, 2, 0, 0], dtype=int32)

In [ ]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0,'Cluster labels',kmeans.labels_)
tor_merged = df

# merge torronto_grouped with df1 to add latitude/longitude for each neighborhood
tor_merged = tor_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

tor_merged.head() # check the last columns!

,Borough,Neighbourhood,Latitude,Longitude,Cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North West Delhi,Adarsh Nagar,28.614192,77.071541,2.0,Indian Restaurant,Yoga Studio,Flea Market,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop
1,North West Delhi,Ashok Vihar,28.699453,77.184826,0.0,Farm,Yoga Studio,Flower Shop,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop
1,North West Delhi,Ashok Vihar,28.699453,77.184826,0.0,Hotel Bar,Yoga Studio,Flea Market,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop
1,North West Delhi,Ashok Vihar,28.699453,77.184826,0.0,Health & Beauty Service,Yoga Studio,Flower Shop,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop
1,North West Delhi,Ashok Vihar,28.699453,77.184826,0.0,Bakery,Yoga Studio,Food,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court


## Visualizing the resulting clusters

In [ ]:
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors

address = 'Delhi, IN'
geolocator = Nominatim(user_agent="abc")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Delhi are {}, {}.'.format(latitude, longitude))

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k_clusters)
ys = [i + x + (i*x)**2 for i in range(k_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

import math
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tor_merged['Latitude'], tor_merged['Longitude'],tor_merged['Neighbourhood'], tor_merged['Cluster labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    if (math.isnan(cluster)!=True):
      folium.CircleMarker(
          [lat, lon],
          radius=5,
          popup=label,
          color=rainbow[int(cluster)-1],
          fill=True,
          fill_color=rainbow[int(cluster)-1],
          fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

The geograpical coordinate of Delhi are 28.65035795, 77.18463072776879.


## Result and Discussion:<a name="results"></a>
The objective of this business problem was to identify a suitable location for the stakeholders to open a restaurant where they can gain huge profit and can survive the market. Here in the analysis, we found the 10 most common venues in neighbourhood in Delhi and with  the help of KMeans clustering algorithm we made neighburhood clusters of Delhi and visualized in the form of iteractive map to show us this venues and in a way helping the stakeholders to find a good place to set up their restaurant.

## Conclusion<a name="conclusion"></a>

Through the analysis of neighbourhood data of delhi we found out the 100 top venues of neighbourhoods and after normalizing our data we found the 10 most common venues that are good and profitable for opening a restaurant.